Alright — here’s a **detailed, in-depth note** covering **every concept mentioned in your transcript** on the topic **pytest** under **AppDev**.
I’ve structured it so it matches the learning outcomes you gave, with **clear breakdowns, examples, and explanations**.

---

# **Pytest – Modern Application Development**

## **1. Introduction to pytest**

* **Definition**:
  Pytest is a Python framework for testing code, designed to make writing and running tests easier.
  It works well for:

  * **Small unit tests** (testing individual functions)
  * **Larger functional/integration tests** (testing across multiple components)

* **Opinionated Framework**:

  * Pytest follows **specific conventions** and defaults for identifying tests.
  * If you follow these conventions, pytest automatically discovers and runs tests without extra configuration.
  * Advantage: Reduces misunderstandings between developers about how tests should be structured.
  * Disadvantage: Some developers may dislike being forced into a particular style.

* **Features**:

  * **Automatic test discovery** (finds test files/functions without manual setup).
  * **Test fixtures**: Setup and teardown logic that can be reused across multiple tests.
  * **Monkeypatching** support: Temporarily altering code behavior for testing (not covered in detail here).
  * **Easy failure reporting**: Shows exact mismatches between expected and actual values.
  * Can handle **environment setup and teardown** for tests.

* **Comparison with `unittest`**:

  * Python's built-in testing framework is `unittest`.
  * Pytest is an **alternative** with additional convenience features.
  * Pytest can still run tests written in `unittest`.

---

## **2. Basic pytest Example**

### Example 1 – Simple Function Test

```python
# test_sample.py
def func(x):
    return x + 1

def test_answer():
    assert func(3) == 5  # This will fail
```

**Explanation**:

* **func(x)** adds 1 to x.
* Test function **`test_answer`** uses `assert` to check if `func(3)` equals 5.
* Since `func(3)` returns `4`, test fails.

**Key Points**:

* **`assert`** in Python:

  * If condition is `True`: test passes.
  * If condition is `False`: raises `AssertionError`, test fails.
* Pytest output shows:

  * Number of tests run
  * Which tests failed
  * Detailed mismatch between expected and actual output.

---

## **3. Testing for Exceptions**

Sometimes, a function should **raise** an exception for invalid cases.

### Example 2 – Checking Exceptions

```python
import pytest

def f():
    raise SystemExit(1)

def test_mytest():
    with pytest.raises(SystemExit):
        f()
```

**Explanation**:

* `pytest.raises(ExceptionType)` ensures that running the given code **raises** the specified exception.
* Useful for validating error-handling code (e.g., accessing an array out-of-bounds).

---

## **4. Temporary Files and Directories in pytest**

* Many tests require **temporary files/directories**.
* Pytest can create them automatically and remove them after tests.

### Example 3 – Temp Directory

```python
def test_needsfiles(tmpdir):
    print(tmpdir)
    assert 0
```

* **`tmpdir`**: Special pytest fixture that creates a temp directory.
* `assert 0` ensures test fails (for demonstration).
* Output includes the generated temp directory path.
* **Advantage**: Ensures test files don’t overwrite or delete existing data.

---

## **5. Test Fixtures**

* **Definition**: A way to define setup and teardown logic **outside** of test functions.
* Fixtures:

  * Prepare test data or environment before a test runs.
  * Clean up after the test.
  * Can be reused across multiple tests.

### Example 4 – Fixture for a List

```python
import pytest

@pytest.fixture
def setup_list():
    return ["apple", "banana"]

def test_apple(setup_list):
    assert "apple" in setup_list

def test_banana(setup_list):
    assert "banana" in setup_list

def test_mango(setup_list):
    assert "mango" in setup_list  # Will fail
```

**How it works**:

1. The fixture function `setup_list()` is decorated with `@pytest.fixture`.
2. Tests declare a parameter named `setup_list`; pytest runs the fixture before the test and passes its return value.
3. Each test gets its own fresh copy of the fixture data.

**Benefits**:

* Avoids duplicate setup code.
* Changes in one test’s fixture data do not affect others.

---

## **6. Pytest Test Discovery Conventions**

Pytest automatically finds tests based on naming patterns:

* **Test file names**:

  * `test_*.py` or `*_test.py`
* **Test function/method names**:

  * Start with `test`
* **Test classes**:

  * Class name starts with `Test`
  * Methods inside start with `test`

**Search process**:

* Starts in **current directory** or from `testpaths` in config.
* Recursively searches subdirectories.

---

## **7. Testing Flask Applications with pytest**

### **7.1. Client Fixture for Flask**

Flask provides a `test_client()` that can simulate HTTP requests.

**Example – Client Fixture**

```python
import os
import tempfile
import pytest
from flaskr import create_app, init_db

@pytest.fixture
def client():
    db_fd, db_path = tempfile.mkstemp()
    app = create_app({
        'TESTING': True,
        'DATABASE': db_path
    })

    with app.app_context():
        init_db()

    yield app.test_client()

    os.close(db_fd)
    os.unlink(db_path)
```

**Explanation**:

* Creates a **temporary SQLite database** for testing.
* Sets `TESTING=True` to enable Flask’s testing mode.
* `yield` returns the test client to the test function, then cleanup runs after the test.

---

### **7.2. Using the Flask Client Fixture**

Example: Testing an empty database state

```python
def test_empty_db(client):
    rv = client.get('/')
    assert b'No entries here so far' in rv.data
```

* **`client.get('/')`**: Sends a GET request to the root URL.
* Checks that the returned page contains the text `"No entries here so far"`.

---

## **8. Testing Login and Other Features**

### **8.1. Helper Functions for Login/Logout**

```python
def login(client, username, password):
    return client.post('/login', data=dict(
        username=username,
        password=password
    ), follow_redirects=True)

def logout(client):
    return client.get('/logout', follow_redirects=True)
```

* **Helper functions** are **not** test cases (do not start with `test`).
* Use `follow_redirects=True` so that after login/logout, the test follows the redirect to the final page.

---

### **8.2. Test Cases for Login**

```python
def test_login_logout(client):
    rv = login(client, 'admin', 'default')
    assert b'You were logged in' in rv.data

    rv = logout(client)
    assert b'You were logged out' in rv.data

    rv = login(client, 'invalid', 'default')
    assert b'Invalid username' in rv.data

    rv = login(client, 'admin', 'wrong')
    assert b'Invalid password' in rv.data
```

* Covers:

  * Successful login
  * Successful logout
  * Invalid username case
  * Invalid password case

---

## **9. Automated Evaluation with pytest**

* Pytest can be used to **automatically grade** student submissions.
* Example: Checking if required files exist.

```python
def test_contact_html_exists():
    assert os.path.exists('contact.html')

def test_resume_html_exists():
    assert os.path.exists('resume.html')
```

* If any file is missing, test fails automatically.

---

## **10. Summary**

* **Why automated testing?**

  * Ensures confidence in design.
  * Detects regressions (features that break after changes).
  * Reduces manual testing effort.
* **Caution**:

  * High test coverage ≠ bug-free code.
  * Continuous testing is essential for system stability.

---

✅ **Learning Outcomes Achieved**:

1. **Understanding pytest** as a Python testing framework and how it simplifies both unit and functional testing.
2. **Test fixtures** explained with examples and their role in setup/teardown.
3. **Pytest conventions** for naming and discovery.
4. **Testing Flask apps** using pytest, including database setup with fixtures.
5. **Testing login/logout** flows using helper functions and simulated HTTP requests.
6. **Automated evaluation** use case for checking student submissions.